<a href="https://colab.research.google.com/github/rodrigogalvaopatriota/AHK_AUTOHOTKEY/blob/master/short_Fine_tune_TrOCR_on_IAM_Handwriting_Database_using_native_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

In [2]:
!pip install -q datasets jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
from PIL import Image, UnidentifiedImageError

path = '/content/drive/MyDrive/TrOCR/img_geral/img_crop/'
data = []
files = os.listdir(path)
print(len(files))

# Função para verificar a validade da imagem e excluir se corrompida
def check_and_remove_invalid_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()  # Verifica a integridade da imagem
        return True
    except (UnidentifiedImageError, OSError) as e:
        print(f"Imagem corrompida ou inválida: {image_path}")
        os.remove(image_path)  # Exclui a imagem corrompida
        return False

for file in files:
    path_file = path + file

    # Verificar se a imagem é válida
    if not check_and_remove_invalid_image(path_file):
        continue  # Pula a imagem se ela for inválida e já foi excluída

    if '1756183' in file:
        print(f'encontrou: {file}')

    if file.endswith('.jpg'):
        text = file.replace('.jpg', '')
    elif file.endswith('.png'):
        text = file.replace('.png', '')

    # Adiciona o caminho e o texto ao DataFrame
    file_name = path + file
    data.append([file_name, text])

# Criar DataFrame e salvar em um arquivo CSV
df = pd.DataFrame(data, columns=['file_name', 'text'])
df['file_name'] = df['file_name'].astype(str)
df['text'] = df['text'].astype(str)

df.to_csv('/content/drive/MyDrive/TrOCR/result_concat.csv', sep=';', index=False)


2661


## Prepare data

We first download the data. Here, I'm just using the IAM test set, as this was released by the TrOCR authors in the unilm repository. It can be downloaded from [this page](https://github.com/microsoft/unilm/tree/master/trocr).

Let's make a [regular PyTorch dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). We first create a Pandas dataframe with 2 columns. Each row consists of the file name of an image, and the corresponding text.

In [ ]:
#https://fki.tic.heia-fr.ch/databases/download-the-iam-handwriting-database
import pandas as pd
#/content/drive/MyDrive/TrOCR/img_geral/result
#df = pd.read_fwf('/content/drive/MyDrive/TrOCR/Tutorial notebooks/IAM/gt_test.txt', header=None)
#df = pd.read_csv('/content/drive/MyDrive/TrOCR/result_concat.csv',sep=';', header=None,skiprows=1,dtype=str)
df = pd.read_csv('/content/drive/MyDrive/TrOCR/result_concat.csv',sep=';',dtype=str)
#df = pd.read_csv('/content/drive/MyDrive/TrOCR/result_concat.csv',sep=';', dtype={'file_name':str,'text':str
#df.rename(columns={0: "file_name", 1: "text"}, inplace=True)
#del df[2]
# some file names end with jp instead of jpg, let's fix this
df['file_name'] = df['file_name'].apply(lambda x: x + 'g' if x.endswith('jp') else x)
df.head()

,file_name,text
0,/content/drive/MyDrive/TrOCR/img_geral/img_cro...,8172269
1,/content/drive/MyDrive/TrOCR/img_geral/img_cro...,2924475
2,/content/drive/MyDrive/TrOCR/img_geral/img_cro...,2822085
3,/content/drive/MyDrive/TrOCR/img_geral/img_cro...,2888125
4,/content/drive/MyDrive/TrOCR/img_geral/img_cro...,8136732


We split up the data into training + testing, using sklearn's `train_test_split` function.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import numpy as np
import os

# Definindo a semente para reprodutibilidade
seed = 42
random.seed(seed)
np.random.seed(seed)

# Caminho dos arquivos de treino e teste
test_file_path = '/content/drive/MyDrive/TrOCR/test_df.csv'
train_file_path = '/content/drive/MyDrive/TrOCR/train_df.csv'

# Verificando se os arquivos já existem
if os.path.exists(test_file_path) and os.path.exists(train_file_path):
    print('Vai ler os dataframes treino e teste.')
    # Lendo os arquivos CSV, com separador ';'
    train_df = pd.read_csv(train_file_path, sep=';',dtype=str)
    test_df = pd.read_csv(test_file_path, sep=';',dtype=str)
    # Resetando os índices para começar do zero
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
else:
    print('Vai gerar os dataframes treino e teste.')
    # Dividindo os dados em treino e teste com 80% para treino e 20% para teste
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)
    # Resetando os índices para começar do zero
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)

    # Salvando os dataframes gerados como arquivos CSV
    train_df.to_csv(train_file_path, sep=';', index=False)
    test_df.to_csv(test_file_path, sep=';', index=False)


Vai ler os dataframes treino e teste.


Each element of the dataset should return 2 things:
* `pixel_values`, which serve as input to the model.
* `labels`, which are the `input_ids` of the corresponding text in the image.

We use `TrOCRProcessor` to prepare the data for the model. `TrOCRProcessor` is actually just a wrapper around a `ViTFeatureExtractor` (which can be used to resize + normalize images) and a `RobertaTokenizer` (which can be used to encode and decode text into/from `input_ids`).

In [5]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        print(f'root_dir: {root_dir}')
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)

        img_path = os.path.join(self.root_dir, file_name)
        #print(f"Trying to open image at path: {img_path}")
        image = Image.open(img_path).convert("RGB")


        #image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

Let's initialize the training and evaluation datasets:

In [6]:
from transformers import TrOCRProcessor
import os
path = os.getcwd()
full_path = os.path.join(path, 'drive','MyDrive','TrOCR','img_geral','img_crop')
print(f'full_path: {full_path}')
#/content/drive/MyDrive/TrOCR/img_geral/
#/content/drive/MyDrive/TrOCR/file_train_08_11
#microsoft/trocr-base-handwritten
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

train_dataset = IAMDataset(root_dir=full_path,
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir=full_path,
                           df=test_df,
                           processor=processor)

full_path: /content/drive/MyDrive/TrOCR/img_geral/img_crop


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

root_dir: /content/drive/MyDrive/TrOCR/img_geral/img_crop
root_dir: /content/drive/MyDrive/TrOCR/img_geral/img_crop


In [7]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))
print(f'sum: {len(train_dataset)+len(eval_dataset)}')

Number of training examples: 2128
Number of validation examples: 533
sum: 2661


Let's verify an example from the training dataset:

We can also check the original image and decode the labels:

Let's create corresponding dataloaders:

In [27]:
from torch.utils.data import DataLoader
#era 4
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=2)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#
print(device)

cuda


## Train a model

Here, we initialize the TrOCR model from its pretrained weights. Note that the weights of the language modeling head are already initialized from pre-training, as the model was already trained to generate text during its pre-training stage. Refer to the paper for details.

In [29]:
from transformers import VisionEncoderDecoderModel,AutoTokenizer,TrOCRProcessor
import torch
from torch.nn import Dropout





torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#file_model = '/content/drive/MyDrive/TrOCR/file_train/train_loss_train_3_10_loss_validation_3_79'
#file_model = '/content/drive/MyDrive/TrOCR/file_train/train_loss_train_2_86_loss_validation_3_80'
#file_train_total_11_11_2
#/content/drive/MyDrive/TrOCR/file_train/overfitting_1
file_model = 'microsoft/trocr-large-handwritten'
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained(file_model)

# Ajustar o dropout no encoder (ViTModel)
model.encoder.config.hidden_dropout_prob = 0.2  # Dropout para camadas ocultas
model.encoder.config.attention_probs_dropout_prob = 0.2  # Dropout para probabilidades de atenção

# Ajustar o dropout no decoder
model.decoder.config.dropout = 0.2  # Dropout para camadas do decoder

# Atualizar os parâmetros no modelo
#model.encoder = model.encoder.from_config(model.encoder.config)
#model.decoder = model.decoder.from_config(model.decoder.config)

# Confirmar as alterações
print("Encoder Dropout (hidden):", model.encoder.config.hidden_dropout_prob)
print("Encoder Dropout (attention):", model.encoder.config.attention_probs_dropout_prob)
print("Decoder Dropout:", model.decoder.config.dropout)



tokenizer = AutoTokenizer.from_pretrained("microsoft/trocr-large-handwritten")

# Congela todas as camadas do encoder
#for param in model.encoder.parameters():
    #param.requires_grad = False

# Alternativamente, você pode congelar camadas específicas do decoder ou encoder
# Exemplo: congelando apenas algumas camadas do decoder
#for layer in model.decoder.layers[:6]:  # congelando as primeiras 6 camadas do decoder
    #for param in layer.parameters():
        #param.requires_grad = False


#try this after save model:
#model = TrOCRForCausalLM.from_pretrained("/content/drive/MyDrive/TrOCR/file_train/")
#or
#model = VisionEncoderDecoderModel.from_pretrained("/content/drive/MyDrive/TrOCR/file_train/")
import gc # Import garbage collector

gc.collect() # Collect garbage
torch.cuda.empty_cache() # Empty cuda cache

model.to(device)
print(device)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

Encoder Dropout (hidden): 0.2
Encoder Dropout (attention): 0.2
Decoder Dropout: 0.2
cuda


Importantly, we need to set a couple of attributes, namely:
* the attributes required for creating the `decoder_input_ids` from the `labels` (the model will automatically create the `decoder_input_ids` by shifting the `labels` one position to the right and prepending the `decoder_start_token_id`, as well as replacing ids which are -100 by the pad_token_id)
* the vocabulary size of the model (for the language modeling head on top of the decoder)
* beam-search related parameters which are used when generating text.

In [30]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 10
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
#model.config.length_penalty = 2.0
model.config.length_penalty = 1.0
model.config.num_beams = 5

In [31]:
import torch

gc.collect() # Collect garbage
torch.cuda.empty_cache() # Empty cuda cache




model.to(device)
print(device)

cuda


In [33]:
#empty memory
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
!pip install datasets --upgrade

We will evaluate the model on the Character Error Rate (CER), which is available in HuggingFace Datasets (see [here](https://huggingface.co/metrics/cer)).

In [ ]:
import random
from tqdm.notebook import tqdm
from transformers import GenerationConfig
from transformers import AdamW, AutoTokenizer, GenerationConfig
from PIL import ImageFile
import torch
import os
from torch.optim.lr_scheduler import StepLR

path_make_dir_validation = '/content/drive/MyDrive/TrOCR/file_train/validation/'
path_make_dir = '/content/drive/MyDrive/TrOCR/file_train/train/'
os.makedirs(path_make_dir, exist_ok=True)

ImageFile.LOAD_TRUNCATED_IMAGES = True
#optimizer = AdamW(model.parameters(), lr=1e-4)
optimizer = AdamW(model.parameters(), lr=1e-8,weight_decay = 0.1)
#optimizer = AdamW(model.parameters(), lr=1e-10,weight_decay=1e-2)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduz a LR a cada 10 épocas

i = 0
for epoch in range(100):  # loop over the dataset multiple times
    # Train
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
        # Move inputs to the device
        for k, v in batch.items():
            batch[k] = v.to(device)

        # Forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

        if i % 50 == 0:
            model.save_pretrained(path_make_dir)
        i += 1

    print(f"Train Loss after epoch {epoch}:", train_loss / len(train_dataloader))

    #Evaluation on test set
    model.eval()
    valid_loss = 0.0
    valid_accuracy = 0.0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            # Move batch to the device
            for k, v in batch.items():
                batch[k] = v.to(device)

            # Forward pass
            outputs = model(**batch)
            loss = outputs.loss
            valid_loss += loss.item()

            #Generate predictions
            generated_outputs = model.generate(batch["pixel_values"])
            predicted_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in generated_outputs]
            target_texts = [tokenizer.decode(label[label != -100], skip_special_tokens=True) for label in batch["labels"]]

            # Calculate accuracy
            correct_predictions = sum([1 for pred, target in zip(predicted_texts, target_texts) if pred == target])
            valid_accuracy += correct_predictions
            total_samples += len(target_texts)
    scheduler.step()
    # Final accuracy and loss on validation set
    valid_accuracy /= total_samples
    valid_loss /= len(eval_dataloader)
    print(f"Validation Accuracy after epoch {epoch}:", valid_accuracy)
    print(f"Validation Loss after epoch {epoch}:", valid_loss)
    #epoch 5 validation loss 3,69


  0%|          | 0/1064 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 10, 'early_stopping': True, 'num_beams': 5, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
#model.save_pretrained("/content/drive/MyDrive/TrOCR/model_trained")
model.save_pretrained(".")

## Inference

Note that after training, you can easily load the model using the .`from_pretrained(output_dir)` method.

For inference on new images, I refer to my inference notebook, that can also be found in my [Transformers Tutorials repository](https://github.com/NielsRogge/Transformers-Tutorials) on Github.